# THIS IS WEEK 3 DATA SCIENTIST CAPSTONE PROJECT

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')



Libraries imported.


#Use pandas read_html to grab the first table and put it in dataframe 'hood'

In [ ]:
hood = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)[0]

In [4]:
hood.columns = (['Postal Code','Borough','Neighborhood'])
print('Raw Frame')
print(hood.shape)
print(hood.head(10))


Raw Frame
(288, 3)
  Postal Code           Borough      Neighborhood
0         M1A      Not assigned      Not assigned
1         M2A      Not assigned      Not assigned
2         M3A        North York         Parkwoods
3         M4A        North York  Victoria Village
4         M5A  Downtown Toronto      Harbourfront
5         M5A  Downtown Toronto       Regent Park
6         M6A        North York  Lawrence Heights
7         M6A        North York    Lawrence Manor
8         M7A      Queen's Park      Not assigned
9         M8A      Not assigned      Not assigned


In [5]:
hood = hood[hood.Borough != 'Not assigned']

#Drop the unassigned borough rows

In [6]:
print('Dropped unassigned boroughs')
print(hood.shape)
print(hood.head(10))


Dropped unassigned boroughs
(211, 3)
   Postal Code           Borough      Neighborhood
2          M3A        North York         Parkwoods
3          M4A        North York  Victoria Village
4          M5A  Downtown Toronto      Harbourfront
5          M5A  Downtown Toronto       Regent Park
6          M6A        North York  Lawrence Heights
7          M6A        North York    Lawrence Manor
8          M7A      Queen's Park      Not assigned
10         M9A         Etobicoke  Islington Avenue
11         M1B       Scarborough             Rouge
12         M1B       Scarborough           Malvern


#Convert neighborhoods to borough where the neighborhood is unassigned

In [7]:
hood['Neighborhood'][hood.Neighborhood == 'Not assigned'] = hood.Borough
print('Convert unassigned neighborhoods to borough name')


Convert unassigned neighborhoods to borough name


In [8]:
print(hood.shape)
print(hood.head(10))


(211, 3)
   Postal Code           Borough      Neighborhood
2          M3A        North York         Parkwoods
3          M4A        North York  Victoria Village
4          M5A  Downtown Toronto      Harbourfront
5          M5A  Downtown Toronto       Regent Park
6          M6A        North York  Lawrence Heights
7          M6A        North York    Lawrence Manor
8          M7A      Queen's Park      Queen's Park
10         M9A         Etobicoke  Islington Avenue
11         M1B       Scarborough             Rouge
12         M1B       Scarborough           Malvern


#Group the PostalCode and Borough columns

In [9]:
hood = hood.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print('Group by Postal and Borough and combine neighborhoods')
print(hood.shape)
print(hood.head()) 


Group by Postal and Borough and combine neighborhoods
(103, 3)
  Postal Code      Borough                            Neighborhood
0         M1B  Scarborough                          Rouge, Malvern
1         M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae


In [14]:
hood.loc[hood['Neighborhood']=="Not assigned",'Neighborhood']=hood.loc[hood['Neighborhood']=="Not assigned",'Borough']

In [23]:
print(hood.shape)
print(hood.head())

(103, 3)
  Postal Code      Borough                            Neighborhood
0         M1B  Scarborough                          Rouge, Malvern
1         M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae


In [28]:
hood.head(200)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [29]:
print(hood.shape)

(103, 3)


#Second Part Starts Here for Lat/Long

In [33]:
filename = "http://cocl.us/Geospatial_data"

In [34]:
headers = ["Postal Code","Latitude","Longitude"]

In [35]:
df = pd.read_csv(filename, names = headers)

In [37]:
df.set_index ('Postal Code').join(hood.set_index('Postal Code'))

,Latitude,Longitude,Borough,Neighborhood
Postal Code,,,,
Postal Code,Latitude,Longitude,NaN,NaN
M1B,43.8066863,-79.1943534,Scarborough,"Rouge, Malvern"
M1C,43.7845351,-79.1604971,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,43.7635726,-79.1887115,Scarborough,"Guildwood, Morningside, West Hill"
M1G,43.7709921,-79.2169174,Scarborough,Woburn
M1H,43.773136,-79.2394761,Scarborough,Cedarbrae
M1J,43.7447342,-79.2394761,Scarborough,Scarborough Village
M1K,43.7279292,-79.2620294,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,43.7111117,-79.2845772,Scarborough,"Clairlea, Golden Mile, Oakridge"
